# Part 1

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:
```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```
Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
Otherwise, the seat's state does not change.
Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:
```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:
```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:
```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##

#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##

#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```
At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [1]:
def get_input():
    with open('inputs/11') as f:
        return [list(r) for r in f.read().splitlines()]
    
def list_neighbors(row, col, all_seats):
    neighbors = []
    neighboring_rows = [row]
    neighboring_cols = [col]
    for i in (-1, 1):
        r = row + i
        c = col + i
        if 0 <= r and r < len(all_seats):
            neighboring_rows.append(r)
        if 0 <= c and c < len(all_seats[0]):
            neighboring_cols.append(c)
    for r in neighboring_rows:
        for c in neighboring_cols:
            if not (r == row and c == col):
                n = all_seats[r][c]
                neighbors.append(n)
    return neighbors

def apply_rule_to_seat(row, col, all_seats):
    seat = all_seats[row][col]
    num_occupied_neighbors = sum(n == '#' for n in list_neighbors(row, col, all_seats))
    if seat == 'L' and num_occupied_neighbors == 0:
        return '#'
    if seat == '#' and num_occupied_neighbors >= 4:
        return 'L'
    return seat

def apply_rule_to_row(row, all_seats):
    num_cols = len(all_seats[0])
    return [apply_rule_to_seat(row, col, all_seats) for col in range(num_cols)]

def apply_rule(seats):
    new_seats = []
    num_rows = len(seats)
    for row in range(num_rows):
        new_row = apply_rule_to_row(row, seats)
        new_seats.append(new_row)
    return new_seats

def count_occupied(seats):
    res = 0
    for row in seats:
        res += sum(seat == '#' for seat in row)
    return res    

def result1():
    # Avoid infinite loop
    loop = 0
    current_seats = get_input()
    while True:
        next_seats = apply_rule(current_seats)
        if current_seats == next_seats:
            break
        else:
            current_seats = next_seats
            loop += 1
            if loop > 1000:
                print('This takes too many loops.')
                return
    return count_occupied(current_seats)

result1()

2211

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:
```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:
```
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:
```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:
```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#

#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```
Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [2]:
def _is_there_visible_occupied_in_direction(row, col, all_seats, direction):
    step_row, step_col = direction
    r = row + step_row
    c = col + step_col
    while is_valid_seat(r, c, all_seats):
        next_seat = all_seats[r][c]
        if next_seat == 'L':
            return False
        if next_seat == '#':
            return True
        r += step_row
        c += step_col
    return False
   
def is_valid_seat(row, col, all_seats):
    return 0 <= row and row < len(all_seats) and 0 <= col and col < len(all_seats[0])

def count_visible_occupied(row, col, all_seats):
    directions = [(i, j) for i in (-1, 0, 1) for j in (-1, 0, 1) if i != 0 or j != 0]
    return sum(_is_there_visible_occupied_in_direction(row, col, all_seats, d) for d in directions)

def apply_rule2_to_seat(row, col, all_seats):
    seat = all_seats[row][col]
    num_visible_occupied = count_visible_occupied(row, col, all_seats)
    if seat == 'L' and num_visible_occupied == 0:
        return '#'
    if seat == '#' and num_visible_occupied >= 5:
        return 'L'
    return seat

def apply_rule2_to_row(row, all_seats):
    num_cols = len(all_seats[0])
    return [apply_rule2_to_seat(row, col, all_seats) for col in range(num_cols)]

def apply_rule2(seats):
    new_seats = []
    num_rows = len(seats)
    for row in range(num_rows):
        new_row = apply_rule2_to_row(row, seats)
        new_seats.append(new_row)
    return new_seats

def result2():
    # Avoid infinite loop
    loop = 0
    current_seats = get_input()
    while True:
        next_seats = apply_rule2(current_seats)
        if current_seats == next_seats:
            break
        else:
            current_seats = next_seats
            loop += 1
            if loop > 1000:
                print('This takes too many loops.')
                return

    return count_occupied(current_seats)

result2()

1995